<h2 align = "center"> Ensemble</h2>

### Some important terms

* Bagging, Random Forest, Boosting
* For bagging, we end up with having 63% of unique observations
* To measure the confidence of the classification, we can either use margin (the difference between the majority vote and the minority vote
* In a real-world situation, we generally want to have a much larger test set to estimate our unseen accuracy. In fact, because of this, we are more inclined to believe our outof-bag accuracy measurement, which is done over a larger number of observations and averaged over many models.
* Adaboost is basically update the coefficient weight for each weak learner, update the weights for each data points all based on the weighted classification error and the final model is the sign of the sum of all the weak learners

### Bagging

### Bagging 

  Strenghts     | Weakness
  ------------- | -------------
  1) Smoothens the overall output and reduce the bias when the target function is smooth| Sampling<br/> 1) sampling might have ignored the level in the categorical variables with very few observations <br/> 2) Unbalanced sample will always favor the majority one <br/>3) High Leverage points bagging performance depends on how often these particular observations are sampled <br/> Bagging: <br/>4) Not truly independent (the same set of the features) 

In [1]:
library(ipred)

In [ ]:
baggedTree = bagging(y~., data = , nbagg = 100, coob = T) # 

In [20]:
setwd("E:/Personal/InterviewQuestion/Rscripts")
bnote = read.table("data_banknote_authentication.txt", sep = ",")
colnames(bnote) = c("waveletVar", "waveletSkew", "waveletCurt", "entropy","class")
#bnote$class = as.factor(bnote$class)

In [3]:
head(bnote)

,waveletVar,waveletSkew,waveletCurt,entropy,class
1,3.6216,8.6661,-2.8073,-0.44699,1
2,4.5459,8.1674,-2.4586,-1.4621,1
3,3.866,-2.6383,1.9242,0.10645,1
4,3.4566,9.5228,-4.0112,-3.5944,1
5,0.32924,-4.4552,4.5718,-0.9888,1
6,4.3684,9.6718,-3.9606,-3.1625,1


In [4]:
n = dim(bnote)[1]

### Construct the bootstrapped samples by sampling the index

In [26]:
M = 11
seeds = 1000: (1000+M-1)
sample_vector = sapply(seeds, function(i){
    set.seed(i); return (sample(n,n,replace = T))})

### Create one initial model

In [22]:
train_1glm = function(sample_indices){
    data = bnote[sample_indices,]
    model = glm(class~., data = data, family = binomial("logit"))
    return(model)
}

### Get all the models

In [23]:
allModels = apply(sample_vector,2,train_1glm)

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

### Find unique observations in each data

In [ ]:
 get_1bag <- function(sample_indices) {     
     unique_sample <- unique(sample_indices)
     df <- heart_train[unique_sample, ]
     df$ID <- unique_sample
     return(df)  }

In [ ]:
bags = apply(sample_vectors, 2, get_1bag)

### Construct the prediction model

In [13]:
glm_prediction = function(model, data, model_index){
    colname = paste("prediction", model_index)
    data[colname] = as.numeric(predict(model, data, type = "response") >0.5)
    return(data[,c("ID", colname)], drop = FALSE)
}

[1] 1372

In [ ]:
training_predictions =  mapply(glm_predictions, models, bags, 1 : M, SIMPLIFY = F) 
# mapply is to  applies FUN to the first elements of each ... argument, the second elements, the third elements, and so on.

In [ ]:
rain_pred_df <- Reduce(function(x, y) merge(x, y, by = "ID", all = T), training_predictions)

In [ ]:
train_pred_vote <- apply(train_pred_df[,-1], 1, function(x) as.numeric(mean(x, na.rm = TRUE) > 0.5))

In [ ]:
### Then merge all the data and its prediction to one dataframe

In [24]:
get_1oo_bag <- function(sample_indices) {  unique_sample <- setdiff(1 : n, unique(sample_indices))
                                         df <- bnote[unique_sample,]     
                                         df$ID <- unique_sample
                                         return(df)  }

In [27]:
oo_bags <- apply(sample_vector, 2, get_1oo_bag) # Each list has different size of observations

### Boosting

  Strenghts     | Weakness
  ------------- | -------------
  1) combining the weak learners can help reduce overfitting|  1) Use all training data and progressively to correct each mistake <br/> 2) Symetric loss funtction no distinction that is made in classification between a fp and fn error

In [ ]:
AdaBoostNN <- function(training_data, output_column, M, hidden_units) {   
    require("nnet")   
    models <- list()   
    alphas <- list()   
    n <- nrow(training_data)   
    model_formula <- as.formula(paste(output_column, '~ .', sep = ''))   
    w <- rep((1/n), n)   
    for (m in 1:M) {    model <- nnet(model_formula, data = training_data, size = hidden_units, weights = w)   
                        models[[m]] <- model    
                        predictions <- as.numeric(predict(model,
                                                          training_data[, -which(names(training_data) == output_column)], type = "class")) 
                        errors <- predictions != training_data[, output_column]     
                        error_rate <- sum(w * as.numeric(errors)) / sum(w)    
                         alpha <- 0.5 * log((1 - error_rate) / error_rate)     
                         alphas[[m]] <- alpha     
                         temp_w <- mapply(function(x, y) if (y) { x * exp(alpha) } else { x * exp(-alpha)}, w, errors)     
                                 w <- temp_w / sum(temp_w)   }   
                        return(list(models = models, alphas = unlist(alphas))) }

## Stochastic Gradient Boosting

in every iteration of boosting, we compute a gradient in the direction of the errors that are made by the model trained in the current iteration.

In [ ]:
N <- 1000
X1 <- runif(N)
X2 <- 2*runif(N)
X3 <- ordered(sample(letters[1:4],N,replace=TRUE),levels=letters[4:1])
X4 <- factor(sample(letters[1:6],N,replace=TRUE))
X5 <- factor(sample(letters[1:3],N,replace=TRUE))
X6 <- 3*runif(N) 
mu <- c(-1,0,1,2)[as.numeric(X3)]

SNR <- 10 # signal-to-noise ratio
Y <- X1**1.5 + 2 * (X2**.5) + mu
sigma <- sqrt(var(Y)/SNR)
Y <- Y + rnorm(N,0,sigma)

# introduce some missing values
X1[sample(1:N,size=500)] <- NA
X4[sample(1:N,size=300)] <- NA

data <- data.frame(Y=Y,X1=X1,X2=X2,X3=X3,X4=X4,X5=X5,X6=X6)

# fit initial model
gbm1 <-
gbm(Y~X1+X2+X3+X4+X5+X6,         # formula
    data=data,                   # dataset
    var.monotone=c(0,0,0,0,0,0), # -1: monotone decrease,
                                 # +1: monotone increase,
                                 #  0: no monotone restrictions
    distribution="gaussian",     # For regression model
    n.trees=1000,                # number of trees
    shrinkage=0.05,              # shrinkage or learning rate,(how fast the algorithm is going to learn the parameter before each sequential model)
                                 # 0.001 to 0.1 usually work
    interaction.depth=3,         # 1: additive model, 2: two-way interactions, etc.
    bag.fraction = 0.5,          # subsampling fraction, 0.5 is probably best
    train.fraction = 0.5,        # fraction of data for training,
                                 # first train.fraction*N used for training
    n.minobsinnode = 10,         # minimum total weight needed in each node
    cv.folds = 3,                # do 3-fold cross-validation
    keep.data=TRUE,              # keep a copy of the dataset with the object
    verbose=FALSE,               # don't print out progress
    n.cores=1)                   # use only a single core (detecting #cores is
                                 # error-prone, so avoided here)

# check performance using an out-of-bag estimator
# OOB underestimates the optimal number of iterations
best.iter <- gbm.perf(gbm1,method="OOB") # Get the best iteration
print(best.iter)

# check performance using a 50% heldout test set
best.iter <- gbm.perf(gbm1,method="test")
print(best.iter)

## Random Forest (see the conversion example in take home) 

  Strenghts     | Weakness
  ------------- | -------------
  1) Feature sampling allows us to consider input features that are successful in splitting the data for only a small range of the target variable. These locally relevant features are rarely chosen without the sampling constraint because we usually prefer features that form good overall splits of the data at a given node in the tree. <br/> 2)sampling input features is useful when we have correlated input features. Regular tree construction tends to favor only one of the features from a correlated set while ignoring the rest despite the fact that the resulting splits from even highly correlated features are not exactly the same.<br/> 3) , this model is a good choice when the number of features exceeds the number of observations. <br/> 4) the sampling process mitigates the cost of constructing a large number of trees </br> 5) Can handle noisy or missing outliers data as well as categorical, continuous features </br> 6) did feature selection </br>|  1)  Unlike a decision tree, the model is not easily interpretable 2)  May require some work to tune the model to the data

In [1]:
library(randomForest)

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.


In [2]:
library(e1071)

In [ ]:
rf_ranges <- list(ntree = c(500, 1000, 1500, 2000), mtry = 3:8) # Very common to build somewhere between 700 and several thousand trees
#n rees is the number of trees that will be built for the ensemble and mtry is the number of features sampled for use at each node for spliting

In [ ]:
rf_tune <- tune(randomForest, LeagueIndex ~ ., data =  skillcraft_train, ranges = rf_ranges)

In [ ]:
rf_tune$best.parameters 

For random forests, it turns out that we can still measure variable importance scores for the different input features by tallying and keeping track of the reductions in our error function across all the trees in the ensemble. 

In [ ]:
Use importance function to find the most important one 

In [ ]:
randomForest(x, y=NULL, xtest=NULL, ytest=NULL, 
             ntree=500,#How many trees
             mtry=, #How many features for each split
             replace=TRUE, 
             classwt=NULL, 
             
             
#Priors of the classes. Need not add up to one. 
 #The basic idea is to weight classes such that rarely observations are more likely to be selected to the group
 #these values are trasformed in probabilities for sampling training data
             cutoff, 
             strata,
             sampsize = if (replace) nrow(x) else ceiling(.632*nrow(x)),
             nodesize = if (!is.null(y) && !is.factor(y)) 5 else 1,
             maxnodes = NULL,
             importance=FALSE, 
#. MeanDecreaseAccuracy is computed from permuting OOB data: For each tree, the prediction error on the out-of-bag portion of the data is
# recorded (error rate for classification, MSE for regression). Then the same is done after permuting
# each predictor variable. The difference between the two are then averaged over all trees, and normalized
# by the standard deviation of the differences. If the standard deviation of the differences is
# equal to 0 for a variable, the division is not done (but the average is almost always equal to 0 in that
    # case).
# MeanDecreaseGini is the total decrease in node impurities from splitting on the variable, averaged over all trees. 
# For classification, the node impurity is measured by the Gini index. For regression, it
                    # is measured by residual sum of squares.
  
                 localImp=FALSE, nPerm=1,
             proximity, #Should proximity measure among the rows be calculated? 
 #It is like clustering as we assume the observations which fall into the same terminal node are more similar 
#Since observations that have similar x values ‘travel’ the same way on splits more often than values
# with dissimilar values, the co-occurence in terminal nodes is a suitable measure of similarity. Using this logic
# a so called proximity matrix is calculated. 
# It is a n by n matrix where each entry gives the proportion of terminal nodes that two observations are fall together
             oob.prox=proximity,
             norm.votes=TRUE, 
             do.trace=FALSE,
             keep.forest=,
             corr.bias=FALSE,
             keep.inbag=FALSE, ...)